In [2]:
# U+0301 是 COMBINING ACUTE ACCENT，加在 'e' 后得到 'é'，在 Unicode 中，他们被称为“标准等价物”，应用程序应将他们视作相同的字符。但 Python 看到的是不同的码位序列，因此判定二者不同
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [3]:
len(s1), len(s2) 

(4, 5)

In [4]:
s1 == s2

False

这个问题的解决方案是使用 unicodedata.normalize 函数提供的 Unicode 规范化。这个函数的第一个参数位这 4 个字符串中的一个：
* 'NFC'
* 'NFD'
* 'NFKC'
* 'NFKD'

In [5]:
# NFC (Normalization Form C) 使用最少的码位构成等价的字符串
# NFD 把组合字符解成基字符和单独的组合字符
# 这两种规范化方式都能让比较行为符合预期
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301'
len(s1), len(s2)

(4, 5)

In [6]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [7]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [8]:
normalize('NFC', s1) == normalize('NFC', s2)

True

In [9]:
normalize('NFD', s1) == normalize('NFD', s2)

True

In [10]:
# 在另外两个规范化形式 (NFKC 和 NFKD) 的首字母缩写词中，K 表示 compatibility (兼容性)
# 他们对 "兼容字符" 有英雄。虽然 Unicode 的目标是为各个字符提供规范的码位，但为了兼容现有标准，有些字符会出现多次
# 如 "μ" 这个字符，码位是 U+03BC，但同时也有 U+00B5 以便与 latin1 互相转换。所以它是一个兼容字符
# 在 NFKC 和 NFKD 模式中，各个兼容字符会被替换成一个或多个兼容分解字符，即使格式有所损失
# 例："½" 二分之一通过兼容分解后得到的是三个字符序列 "1⁄2"
from unicodedata import normalize, name
half = '½'
normalize('NFKC', half)

'1⁄2'

In [11]:
four_squared = '4²'
normalize('NFKC', four_squared)

'42'

因此 NFKD 与 NFKC 有可能会损失或曲解信息，尽量不要在存储时使用，但它相当适合在查询时使用

大小写折叠
将所有文本变为小写再做其他转换。这个功能由 str.casefold() 方法支持
注意它与 str.lower() 不同